# 2. Preprocesado de texto

In [103]:
# Importar paquetes necesarios
import numpy as np
import pandas as pd
import os

import string
import unicodedata
!pip install stop_words
from stop_words import get_stop_words

import sys
sys.path.append('../..')

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

seed = 17

pd.set_option('display.max_rows', None) # Muestra todas las filas
pd.set_option('display.max_columns', None) # Muestra todas las columnas

You should consider upgrading via the 'C:\Users\Seven\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [104]:
# Datos descargados de http://jmcauley.ucsd.edu/data/amazon/
# ! unzip datasets.zip

import zipfile
with zipfile.ZipFile("datasets.zip", "r") as z:
    z.extractall(".")

In [105]:
datasets_path = 'datasets'
instruments_json = 'reviews_Musical_Instruments_5.json.gz'
df_instruments = pd.read_json(os.path.join(datasets_path, instruments_json), lines = True, compression = 'gzip')

# Subconjunto de 'reviewText'
reviews = df_instruments[['reviewText']]

In [106]:
def get_lemmas_dict(data_path, lemmas_dict_file):
    lemmas_dict = {}
    with open(os.path.join(data_path, lemmas_dict_file), 'r', encoding='utf-8') as f:
        for line in f:
            (key, val) = line.split()
            lemmas_dict[str(val)] = key
    return lemmas_dict

In [107]:
lemmas_path = 'datasets'
english_lemmas = 'lemmatization.txt'

english_lemmas_dict = get_lemmas_dict(lemmas_path, english_lemmas)
sw_list = get_stop_words('english')

In [108]:
# Preprocesado
def text_normalizer(text, lemmas_dict=None, sw_list=None):

    processed_text = []

    if sw_list is None:
        sw_list = []

    # Convertir a minúsculas
    text = text.lower()

    # Eliminar dígitos
    text = ''.join([c for c in text if not c.isdigit()])

    # Eliminar carácteres extraños
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Eliminar puntuación
    table = str.maketrans('', '', string.punctuation)

    # Dividir en sentencias
    sentences = text.split('.')

    for sentence in sentences:
        words = sentence.split(' ')
        for word in words:
            if word not in sw_list:
                if word not in string.punctuation:
                    word = word.translate(table)
                    if lemmas_dict is not None:
                        word = lemmas_dict.get(word, word)
                processed_text.append(word)

    return ' '.join(processed_text)

In [109]:
# Nueva columna con texto procesado
reviews['processed_text'] = reviews['reviewText'].apply(lambda text: text_normalizer(text=text, lemmas_dict=english_lemmas_dict, sw_list=sw_list))

reviews.head()

,reviewText,processed_text
0,"Not much to write about here, but it does exac...",much write here exactly suppose filter pop so...
1,The product does exactly as it should and is q...,product exactly quite affordable realize doubl...
2,The primary job of this device is to block the...,primary job device block breath otherwise prod...
3,Nice windscreen protects my MXL mic and preven...,nice windscreen protect mxl mic prevent pop t...
4,This pop filter is great. It looks and perform...,pop filter great look perform like studio fil...
